In [1]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=750cccf64c8ff7b238ee086ede62145f83a7d173e33ffc98ef26f48524ce9104
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.DataFrame()
read_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/elperiodicoextremadura/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

20134


In [5]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    try:
      coverpage=soup.find_all('div',class_='headline-article')
      title=coverpage[0].find('h1').get_text()
      try:
       description=coverpage[0].find('h2').get_text()
      except:
        description=''
    except:
      title=''
      description=''
    
    try:
      coverpage=soup.find_all('div',class_='article-entry-author')
      time=coverpage[0].find('span',class_='article-author__date').get_text()
    except:
      time=''  

    try:
      coverpage=soup.find_all('div',class_='article-body')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [6]:
get_articles('https://www.elperiodicoextremadura.com/espana/2022/08/05/fiscalia-investiga-denuncia-exjefe-policia-73095079.html')

['La Fiscalía investiga la denuncia de un exjefe de Policía Nacional contra Cospedal',
 'También a el exministro Jorge Fernández Díaz; el ex secretario de Estado de Interior, Francisco Martínez, y el excomisario José Manuel Villarejo por los presuntos delitos de organización criminal, prevaricación, malversación y revelación de secretos',
 '\n                    05·08·22\n            ',
 '\n        La Fiscalía de Barcelona ha abierto una investigación a raíz de la denuncia que presentó el exjefe de la Policía Nacional en Cataluña Narcís Ortega contra la ex secretaria general del PP María Dolores de Cospedal; el exministro Jorge Fernández Díaz; el ex secretario de Estado de Interior, Francisco Martínez, y el excomisario José Manuel Villarejo por los presuntos delitos de organización criminal, prevaricación, malversación y revelación de secretos en la Operación Cataluña.\n\n        A la vez que ha acordado abrir la investigación, el fiscal ha decido enviarla a la Fiscalía de Madrid porqu

In [7]:
pandarallel.initialize(nb_workers= 50, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='Cáceres'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/elperiodicoextremadura/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


article error
https://www.elperiodicoextremadura.com/videos/deportes/2022/03/26/amistoso-espana-albania-64319704.html
article error
https://www.elperiodicoextremadura.com/fotos/espana/2021/05/04/madrid-decide-futuro-elecciones-generales-51131548.html
article error
https://www.elperiodicoextremadura.com/fotos/internacional/2022/07/25/multiples-victimas-tiroteo-personas-hogar-70767768.html
article error
https://www.elperiodicoextremadura.com/ocio/planes/2022/05/26/10-piscinas-naturales-bonitas-espana-66576341.html
article error
https://www.elperiodicoextremadura.com/ocio/planes/2022/05/17/playas-interior-espectaculares-espana-66207036.html
article error
https://www.elperiodicoextremadura.com/fotos/espana/2021/06/02/maria-dolores-cospedal-ignacio-lopez-52543780.html
article error
https://www.elperiodicoextremadura.com/fotos/espana/2022/07/03/princesa-leonor-infanta-sofia-visitan-67945658.html
article error
https://www.elperiodicoextremadura.com/fotos/espana/2022/07/13/segunda-jornada-deba